In [4]:
#copied from starter code
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

import sys
import os

# Import API key
path = os.path.abspath("../Starter_Code/starter_code")
sys.path.append(path)
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.08,81,92,10.73,SH,1688821752
1,1,grytviken,-54.2811,-36.5092,-6.97,81,18,4.71,GS,1688821753
2,2,dolores,-36.3132,-57.6792,8.41,71,100,2.76,AR,1688821691
3,3,prince rupert,54.3161,-130.3201,15.07,91,82,0.35,CA,1688821754
4,4,port-aux-francais,-49.3500,70.2167,1.85,76,100,6.05,TF,1688821754


In [6]:
# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                          frame_width = 800, frame_height = 600, size='Humidity', color='City')

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
filter_temp = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27), :]
filter_clouds = filter_temp.loc[filter_temp['Cloudiness'] == 0, :]
filter_wind = filter_clouds.loc[filter_clouds['Wind Speed'] < 4.5, :]

# Drop any rows with null values
ideal_cities_df = filter_wind.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,holualoa,19.6228,-155.9522,21.39,83,0,1.54,US,1688821771
79,79,statesville,35.7826,-80.8873,26.41,58,0,2.06,US,1688821774
124,124,kailua-kona,19.6406,-155.9956,23.46,83,0,1.54,US,1688821793
136,136,ghanzi,-21.5667,21.7833,23.47,17,0,0.45,BW,1688821799
167,167,namibe,-15.1961,12.1522,22.26,67,0,3.98,AO,1688821809


In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
69,holualoa,US,19.6228,-155.9522,83,
79,statesville,US,35.7826,-80.8873,58,
124,kailua-kona,US,19.6406,-155.9956,83,
136,ghanzi,BW,-21.5667,21.7833,17,
167,namibe,AO,-15.1961,12.1522,67,


In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
holualoa - nearest hotel: Kona Hotel
statesville - nearest hotel: The Kerr House Bed & Breakfast
kailua-kona - nearest hotel: Kona Seaside Hotel
ghanzi - nearest hotel: Ghanzi farmhouse
namibe - nearest hotel: Hotel Chik Chik Namibe
hamilton - nearest hotel: North Vista Manor
vila velha - nearest hotel: Hotel Prainha
prado - nearest hotel: Pousada Recanto do Prado
kolwezi - nearest hotel: Guest House le Manguier
greenville - nearest hotel: Quality Inn
mishelevka - nearest hotel: No hotel found
lewiston orchards - nearest hotel: Holiday Inn Express Lewiston
yuanping - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
69,holualoa,US,19.6228,-155.9522,83,Kona Hotel
79,statesville,US,35.7826,-80.8873,58,The Kerr House Bed & Breakfast
124,kailua-kona,US,19.6406,-155.9956,83,Kona Seaside Hotel
136,ghanzi,BW,-21.5667,21.7833,17,Ghanzi farmhouse
167,namibe,AO,-15.1961,12.1522,67,Hotel Chik Chik Namibe
192,hamilton,US,39.1834,-84.5333,74,North Vista Manor
218,vila velha,BR,-20.3297,-40.2925,67,Hotel Prainha
370,prado,BR,-17.3411,-39.2208,64,Pousada Recanto do Prado
383,kolwezi,CD,-10.7148,25.4667,26,Guest House le Manguier
386,greenville,US,35.6127,-77.3663,93,Quality Inn


In [23]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                          frame_width = 800, frame_height = 600, size='Humidity', color='City',
                                  hover_cols = ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)